In [2]:
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime

from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.applications import VGG16, InceptionV3, ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Model, load_model

import logging

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [3]:
# Carpeta raíz donde están las clases originales
root_dir = "/kaggle/input/hampreprocessed/full"
# Carpeta destino donde quieres la nueva estructura
dest_root = "data/"

# Lista de carpetas que consideras benignas
benign_classes = ["nv", "bkl", "vasc", "df"]

# Lista de splits que quieres procesar
splits = ["train", "test", "val"]

for split in splits:
    src_split = os.path.join(root_dir, split)
    dst_split = os.path.join(dest_root, split)

    # Crear carpeta destino del split
    os.makedirs(dst_split, exist_ok=True)

    # Crear carpeta "benignas" en el destino
    benign_dest = os.path.join(dst_split, "benignas")
    os.makedirs(benign_dest, exist_ok=True)

    # Recorrer todas las clases en el split original
    for cls in os.listdir(src_split):
        cls_path = os.path.join(src_split, cls)
        if not os.path.isdir(cls_path):
            continue

        if cls in benign_classes:
            # Copiar todas las imágenes a la carpeta "benignas"
            for fname in os.listdir(cls_path):
                src = os.path.join(cls_path, fname)
                dst = os.path.join(benign_dest, fname)
                shutil.copy(src, dst)
        else:
            # Crear carpeta de clase en destino
            dst_cls = os.path.join(dst_split, cls)
            os.makedirs(dst_cls, exist_ok=True)

            # Copiar todas las imágenes a su carpeta correspondiente
            for fname in os.listdir(cls_path):
                src = os.path.join(cls_path, fname)
                dst = os.path.join(dst_cls, fname)
                shutil.copy(src, dst)

    print(f"Split {split} procesado en {dst_split}")


Split train procesado en data/train
Split test procesado en data/test
Split val procesado en data/val


In [ ]:
data_dir = "../data/train"
val_dir = "../data/val"

# Augmentación para entrenamiento
datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    brightness_range=[0.8,1.2],
    shear_range=0.2,
    horizontal_flip=True, # Contempla manchas simétricas
)

# Generador de entrenamiento
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

datagen_val = ImageDataGenerator(
    rescale=1./255
)

val_generator = datagen_val.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

print(pd.Series(val_generator.classes).value_counts())
print(pd.Series(train_generator.classes).value_counts())

In [ ]:
labels = train_generator.classes

# Calculamos los pesos
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(labels),
    y=labels
)

# Lo convertimos en diccionario para Keras
class_weights = dict(enumerate(class_weights))
print(class_weights)


## Modelo

In [ ]:
def clasificador_binario(input_shape=(224,224,3), lr=1e-3):
    entrada = Input(shape=input_shape, name='entrada_imagen')

    # Bloque 1
    x = Conv2D(32, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-4))(entrada)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)

    # Bloque 2
    x = Conv2D(64, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)

    # Bloque 3
    x = Conv2D(128, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)

    # Bloque 4 (extra para más capacidad)
    x = Conv2D(256, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)

    # Global pooling en lugar de Flatten (reduce parámetros)
    x = GlobalAveragePooling2D()(x)

    # Capa densa
    x = Dense(128, activation='relu', kernel_regularizer=l2(1e-4))(x)
    x = Dropout(0.2)(x)

    salida = Dense(7, activation='softmax', name='salida_clases')(x)

    modelo = Model(inputs=entrada, outputs=salida, name='cnn_mejorada')
    modelo.compile(optimizer=Adam(learning_rate=lr),
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])
    return modelo



In [5]:
import tensorflow as tf

# Desactiva todas las GPUs
tf.config.set_visible_devices([], 'GPU')


In [ ]:
# Ajustar nivel de logging de TensorFlow
logging.getLogger("tensorflow").setLevel(logging.ERROR)

modelo = clasificador_binario()  

history = modelo.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    class_weight=class_weights
)


C:\Users\alumno\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 409s 2s/step - accuracy: 0.0510 - loss: -976.7233 - val_accuracy: 0.0524 - val_loss: -3173.8054
Epoch 2/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 368s 1s/step - accuracy: 0.0513 - loss: -13526.7090 - val_accuracy: 0.0524 - val_loss: -23161.7871
Epoch 3/20
223/267 ━━━━━━━━━━━━━━━━━━━━ 1:00 1s/step - accuracy: 0.0491 - loss: -45791.3672

In [ ]:
# Current timestamp
timestamp = datetime.datetime.now().strftime("%m_%d_h%H_%M")

# Carpeta donde guardar
save_dir = "../models/classifier"
os.makedirs(save_dir, exist_ok=True)

modelo.save(f"../models/classifier/new_model_{timestamp}.keras")

In [ ]:
modelo.summary()

Model: "cnn_mejorada"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ entrada_imagen (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 28, 28, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ salida_binaria (Dense)          │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,268,165 (4.84 MB)

 Trainable params: 422,401 (1.61 MB)

 Non-trainable params: 960 (3.75 KB)

 Optimizer params: 844,804 (3.22 MB)

## Finetuning de VGG19

In [5]:
def create_transfer_model(base_model_fn, input_shape=(224,224,3), n_classes=4, dropout=0.2, trainable_layers=0):
    base = base_model_fn(
        include_top=False,
        weights="imagenet",
        input_shape=input_shape
    )

    # Congelamos todas las capas primero
    base.trainable = False

    # Si se especifican capas entrenables, las activamos desde el final
    if trainable_layers > 0:
        for layer in base.layers[-trainable_layers:]:
            layer.trainable = True

    x = GlobalAveragePooling2D()(base.output)
    x = Dense(128, activation="relu")(x)
    x = Dense(64, activation="relu")(x)
    x = Dropout(dropout)(x)
    output = Dense(n_classes, activation="sigmoid")(x)

    model = Model(inputs=base.input, outputs=output)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss="binary_crossentropy",
        metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )
    return model


def train_model(model, train_gen, val_gen, epochs=20, class_weights=None):
    callbacks = [
        EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True),
        ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=4)
    ]
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=epochs,
        class_weight=class_weights,
        callbacks=callbacks
    )
    return history

data_dir = "data/train"
val_dir = "data/val"

def get_generators(data_dir, val_dir, preprocess_fn, target_size=(224, 224), batch_size=256):
    datagen = ImageDataGenerator(
        preprocessing_function=preprocess_fn,
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.2,
        brightness_range=[0.8, 1.2],
        shear_range=0.2,
        horizontal_flip=True
    )

    train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
    )

    val_generator = ImageDataGenerator(preprocessing_function=preprocess_fn).flow_from_directory(
        val_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )

    return train_generator, val_generator

train_generator, val_generator = get_generators(data_dir, val_dir, lambda x: x)
print(pd.Series(val_generator.classes).value_counts())
print(pd.Series(train_generator.classes).value_counts())

labels = train_generator.classes  

# Calculamos los pesos
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(labels),
    y=labels
)

# Lo convertimos en diccionario para Keras
class_weights = dict(enumerate(class_weights))
print(class_weights)

Found 8517 images belonging to 4 classes.
Found 496 images belonging to 4 classes.
2    399
3     55
1     26
0     16
Name: count, dtype: int64
2    6855
3     947
1     437
0     278
Name: count, dtype: int64
{0: 7.659172661870503, 1: 4.872425629290618, 2: 0.31061269146608317, 3: 2.248416050686378}


In [6]:
from keras.applications.vgg16 import preprocess_input as vgg16_preprocess
train_generator, val_generator = get_generators(data_dir,val_dir, vgg16_preprocess)

Found 8517 images belonging to 4 classes.
Found 496 images belonging to 4 classes.


In [7]:
vgg_model = create_transfer_model(VGG16, trainable_layers= 4)
train_model(vgg_model, train_generator, val_generator, class_weights=class_weights, epochs=40)

I0000 00:00:1764434972.954333      71 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/40


I0000 00:00:1764434990.266343     161 service.cc:148] XLA service 0x7d47a8807a70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1764434990.267279     161 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1764434990.820828     161 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-11-29 16:50:02.585687: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng11{k2=1,k3=0} for conv (f32[256,64,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f32[256,64,224,224]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kRelu","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]} is taking a while...
2025-11-2

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.4673 - loss: 0.7723 - precision: 0.4469 - recall: 0.3590

2025-11-29 16:53:27.193561: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng11{k2=1,k3=0} for conv (f32[240,64,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f32[240,64,224,224]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kRelu","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]} is taking a while...
2025-11-29 16:53:27.794293: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.600838492s
Trying algorithm eng11{k2=1,k3=0} for conv (f32[240,64,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f32[240,64,224,224]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivat

34/34 ━━━━━━━━━━━━━━━━━━━━ 255s 6s/step - accuracy: 0.4674 - loss: 0.7675 - precision: 0.4483 - recall: 0.3573 - val_accuracy: 0.6210 - val_loss: 0.3877 - val_precision: 0.7183 - val_recall: 0.4677 - learning_rate: 1.0000e-04
Epoch 2/40
34/34 ━━━━━━━━━━━━━━━━━━━━ 155s 5s/step - accuracy: 0.5839 - loss: 0.4723 - precision: 0.6362 - recall: 0.4815 - val_accuracy: 0.7198 - val_loss: 0.2753 - val_precision: 0.7794 - val_recall: 0.6411 - learning_rate: 1.0000e-04
Epoch 3/40
34/34 ━━━━━━━━━━━━━━━━━━━━ 151s 4s/step - accuracy: 0.6392 - loss: 0.3772 - precision: 0.6877 - recall: 0.5804 - val_accuracy: 0.6573 - val_loss: 0.3030 - val_precision: 0.7362 - val_recall: 0.6190 - learning_rate: 1.0000e-04
Epoch 4/40
34/34 ━━━━━━━━━━━━━━━━━━━━ 152s 4s/step - accuracy: 0.6431 - loss: 0.3589 - precision: 0.6944 - recall: 0.5936 - val_accuracy: 0.6532 - val_loss: 0.3272 - val_precision: 0.7620 - val_recall: 0.5746 - learning_rate: 1.0000e-04
Epoch 5/40
34/34 ━━━━━━━━━━━━━━━━━━━━ 152s 4s/step - accuracy: 

In [9]:
timestamp = datetime.datetime.now().strftime("%m_%d_%H:%M")
vgg_model.save(f"vgg16Full_finetuned_{timestamp}.keras")